<a href="https://colab.research.google.com/github/MasterMilkX/BMO_chatbot_prototype/blob/main/bmo_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#scrapper

In [ ]:
#use the microstudio_scrapper to get links.txt

In [ ]:
# !pip install wget
import wget
import zipfile

In [ ]:
# find source 
def author_game(string):
    index = []
    for i, c in enumerate(string):
        if c == '/':
            index.append(i)
    return string[index[2] : index[4]]

# extract ms folder from the zip file
def extract_zip(file_name):
    archive = zipfile.ZipFile(file_name+'.zip')

    for file in archive.namelist():
        if file.startswith('ms'):
            archive.extract(file, 'codes/'+file_name)
        if file.startswith('sprites'):
            archive.extract(file, 'sprites/'+file_name)

In [ ]:
file1 = open('links.txt', 'r')
lines = file1.readlines()

In [ ]:
for line in lines:
    url = line
    url_name = "_".join(author_game(url).split("/")[1:])
    wget.download(url, url_name + '.zip')
    extract_zip(url_name)
    # os.remove(url_name + '.zip')

In [ ]:
!mkdir zips

mkdir: cannot create directory ‘zips’: File exists


In [ ]:
!mv sprites /content/drive/MyDrive/microstudio_games

In [ ]:
!du /content/drive/MyDrive/microstudio_games -h

# data

In [2]:
#path to zip file
!cp "/content/drive/MyDrive/bmo s/microstudio_games.zip" ./

In [3]:
import zipfile
with zipfile.ZipFile("microstudio_games.zip", 'r') as zip_ref:
    zip_ref.extractall("./")

In [4]:
#convert directories to list of source code
import os
def getListOfFiles(dirName):
    listOfFile = os.listdir(dirName)
    allFiles = list()
    for entry in listOfFile:
        fullPath = os.path.join(dirName, entry)
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles 
code_list = getListOfFiles("microstudio_games/codes")

In [5]:
import pandas as pd
df = pd.DataFrame(columns = ["codes"])
for file in code_list:
  with open(file, 'r') as f:
    string = f.read()
    df = df.append({"codes" : string}, ignore_index=True)

# converting into pandas df/csv
df.to_csv('data.csv')

# copy paste train codegen by salesforce

In [ ]:
# !pip install --upgrade pip setuptools
# !pip install torch --extra-index-url https://download.pytorch.org/whl/cu113
# !pip install transformers==4.21.1 datasets==1.16.1 deepspeed==0.7.0
!pip install mpi4py

In [18]:
!deepspeed --num_gpus=1 train_deepspeed.py

[2022-10-08 12:02:02,124] [WARNING] [runner.py:178:fetch_hostfile] Unable to find hostfile, will proceed with training with local resources only.
[2022-10-08 12:02:02,158] [INFO] [runner.py:504:main] cmd = /usr/bin/python3 -u -m deepspeed.launcher.launch --world_info=eyJsb2NhbGhvc3QiOiBbMF19 --master_addr=127.0.0.1 --master_port=29500 train_deepspeed.py
[2022-10-08 12:02:05,360] [INFO] [launch.py:129:main] 0 NV_LIBNCCL_DEV_PACKAGE=libnccl-dev=2.8.4-1+cuda11.2
[2022-10-08 12:02:05,360] [INFO] [launch.py:129:main] 0 NCCL_VERSION=2.8.4-1
[2022-10-08 12:02:05,360] [INFO] [launch.py:129:main] 0 NV_LIBNCCL_PACKAGE_VERSION=2.8.4-1
[2022-10-08 12:02:05,360] [INFO] [launch.py:129:main] 0 NV_LIBNCCL_PACKAGE=libnccl2=2.8.4-1+cuda11.2
[2022-10-08 12:02:05,360] [INFO] [launch.py:129:main] 0 NV_LIBNCCL_DEV_PACKAGE_NAME=libnccl-dev
[2022-10-08 12:02:05,360] [INFO] [launch.py:129:main] 0 NV_LIBNCCL_PACKAGE_NAME=libnccl2
[2022-10-08 12:02:05,360] [INFO] [launch.py:129:main] 0 NV_LIBNCCL_DEV_PACKAGE_VER

In [1]:
import os
import argparse
import random
import math
import mpi4py

from time import time

import numpy as np

import torch

from transformers import AutoConfig, AutoModelForCausalLM

import deepspeed

In [2]:
DEEPSPEED_CONFIG = \
{
    'fp16': {'enabled': True, 'loss_scale': 0, 'loss_scale_window': 1000, 'initial_scale_power': 12, 'hysteresis': 2, 'min_loss_scale': 1},
    'optimizer': {'type': 'AdamW', 'params': {'lr': 1e-05, 'betas': [0.9, 0.999], 'eps': 1e-08, 'weight_decay': 0.0}},
    'scheduler': {'type': 'WarmupLR', 'params': {'warmup_min_lr': 0, 'warmup_max_lr': 1e-05, 'warmup_num_steps': 100}},
    'zero_optimization': {
        'stage': 3,
        'offload_optimizer': {'device': 'cpu', 'pin_memory': False},
        'offload_param': {'device': 'cpu', 'pin_memory': False},
        'overlap_comm': True,
        'contiguous_gradients': True,
        'sub_group_size': 1e9,
        'reduce_bucket_size': 16777216,
        'stage3_prefetch_bucket_size': 15099494.4,
        'stage3_param_persistence_threshold': 40960,
        'stage3_max_live_parameters': 1e9,
        'stage3_max_reuse_distance': 1e9,
        'stage3_gather_fp16_weights_on_model_save': True
    },
    'train_batch_size': 32,
    'train_micro_batch_size_per_gpu': 2,
    'gradient_accumulation_steps': 16,
    'gradient_clipping': 1.0,
    'steps_per_print': 8,
    'wall_clock_breakdown': False,
    'compression_training': {'weight_quantization': {'shared_parameters': {}, 'different_groups': {}}, 'activation_quantization': {'shared_parameters': {}, 'different_groups': {}}, 'sparse_pruning': {'shared_parameters': {}, 'different_groups': {}}, 'row_pruning': {'shared_parameters': {}, 'different_groups': {}}, 'head_pruning': {'shared_parameters': {}, 'different_groups': {}}, 'channel_pruning': {'shared_parameters': {}, 'different_groups': {}}}
}


def create_args(args=argparse.Namespace()):

    args.seed = 42

    args.model = 'Salesforce/codegen-350M-multi'

    args.deepspeed_config = DEEPSPEED_CONFIG

    args.opt_steps_train = 1000

    return args


In [3]:
## train

def train(args):

    #######################
    ## preamble

    set_seed(args.seed)


    #######################
    ## model

    print('initializing model')

    config = AutoConfig.from_pretrained(args.model)
    config.gradient_checkpointing = True
    config.use_cache = False

    model = AutoModelForCausalLM.from_pretrained(args.model, config=config)

    model.train()
    # TODO(enijkamp): we need to set this flag twice?
    model.gradient_checkpointing_enable()


    #######################
    ## deepspeed

    print('initializing deepspeed')

    model_parameters = list(filter(lambda p: p.requires_grad, model.parameters()))
    model_engine, optimizer, _, _ = deepspeed.initialize(config=args.deepspeed_config, model=model, model_parameters=model_parameters)

    torch.cuda.empty_cache()


    #######################
    ## train

    print('starting training')

    input_ids = torch.randint(low=0, high=10, size=[args.deepspeed_config['train_micro_batch_size_per_gpu'], 1024], dtype=torch.int64).cuda()

    for step in range(args.opt_steps_train+1):

        loss = model_engine(input_ids=input_ids, labels=input_ids).loss

        model_engine.backward(loss)
        model_engine.step()

        print(f'{step} {loss:8.3f}')
    return model_engine

In [4]:
########################################################################################################
## preamble

def set_gpus(gpu):
    torch.cuda.set_device(gpu)


def set_seed(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)


def set_cuda(deterministic=True):
    if torch.cuda.is_available():
        torch.backends.cudnn.deterministic = deterministic
        torch.backends.cudnn.benchmark = not deterministic


def get_exp_id(file):
    return os.path.splitext(os.path.basename(file))[0]


def get_output_dir(exp_id):
    import datetime
    t = datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
    output_dir = os.path.join('output/' + exp_id, t)
    return output_dir


def copy_source(file, output_dir):
    import shutil
    shutil.copyfile(file, os.path.join(output_dir, os.path.basename(file)))

In [5]:
# def main():

# preamble
exp_id = get_exp_id("/content/data.csv")
output_dir = get_output_dir(exp_id)

# args
args = create_args()
args.output_dir = output_dir
args.exp_id = exp_id

# output
os.makedirs(args.output_dir, exist_ok=True)
copy_source("/content/data.csv", args.output_dir)

# train
me = train(args=args)

initializing model
initializing deepspeed
[2022-10-08 12:57:41,220] [INFO] [logging.py:68:log_dist] [Rank -1] DeepSpeed info: version=0.7.0, git-hash=unknown, git-branch=unknown
[2022-10-08 12:57:41,224] [INFO] [comm.py:613:init_distributed] Not using the DeepSpeed or dist launchers, attempting to detect MPI environment...
[2022-10-08 12:57:41,662] [INFO] [comm.py:670:mpi_discovery] Discovered MPI settings of world_rank=0, local_rank=0, world_size=1, master_addr=172.28.0.2, master_port=29500
[2022-10-08 12:57:41,664] [INFO] [comm.py:630:init_distributed] Initializing TorchBackend in DeepSpeed with backend nccl
[2022-10-08 12:57:41,675] [WARNING] [config_utils.py:64:_process_deprecated_field] Config parameter stage3_gather_fp16_weights_on_model_save is deprecated use gather_16bit_weights_on_model_save instead
[2022-10-08 12:57:45,217] [INFO] [logging.py:68:log_dist] [Rank 0] DeepSpeed Flops Profiler Enabled: False
Installed CUDA version 11.2 does not match the version torch was compiled

/usr/local/lib/python3.7/dist-packages/transformers/models/codegen/modeling_codegen.py:167: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at  ../aten/src/ATen/native/TensorCompare.cpp:402.)
  attn_weights = torch.where(causal_mask, attn_weights, mask_value)


0    3.805
1    3.805
2    3.805
3    3.805
4    3.805
5    3.805
6    3.805
[2022-10-08 12:58:19,263] [INFO] [timer.py:207:stop] 0/8, RunningAvgSamplesPerSec=1.4251343101927376, CurrSamplesPerSec=1.4708660577895545, MemAllocated=0.21GB, MaxMemAllocated=2.25GB
7    3.805
8    3.805
9    3.805
10    3.805
11    3.805
12    3.805
13    3.805
14    3.805
[2022-10-08 12:58:32,096] [INFO] [timer.py:207:stop] 0/16, RunningAvgSamplesPerSec=1.3179210559302654, CurrSamplesPerSec=0.7086238201651883, MemAllocated=0.11GB, MaxMemAllocated=2.25GB
15    3.805
16    3.027
17    3.027
18    3.027
19    3.027
20    3.027
21    3.027
22    3.027
[2022-10-08 12:58:43,772] [INFO] [timer.py:207:stop] 0/24, RunningAvgSamplesPerSec=1.3369556950075212, CurrSamplesPerSec=1.4982632265549873, MemAllocated=0.21GB, MaxMemAllocated=2.25GB
23    3.027
24    3.027
25    3.027
26    3.027
27    3.027
28    3.027
29    3.027
30    3.027
[2022-10-08 12:58:55,752] [INFO] [timer.py:207:stop] 0/32, RunningAvgSamplesPerSec=1

In [6]:
an = []
for a  in me.children():
  an = a

from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("Salesforce/codegen-350M-multi")
# model = AutoModelForCausalLM.from_pretrained("Salesforce/codegen-350M-multi")

text = "def hello_world():"
input_ids = tokenizer(text, return_tensors="pt").input_ids.cuda()

# an.to_device("cpu")

generated_ids = an.generate(input_ids, max_length=128)
print(tokenizer.decode(generated_ids[0], skip_special_tokens=True))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.7/dist-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


def hello_world():
    print('Hello World!')

hello_world()



In [7]:
text = "draw = function()"
input_ids = tokenizer(text, return_tensors="pt").input_ids.cuda()

# an.to_device("cpu")

generated_ids = an.generate(input_ids, max_length=128)
print(tokenizer.decode(generated_ids[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


draw = function() {
    $(document).on('click', '#' + this.id, function(e) {
      $(this).toggleClass('active');
      $(this).siblings('.active').removeClass('active');
      $(this).siblings('.active').addClass('active');
      $(this).siblings('.active').siblings('.active').removeClass('active');
      $(this).siblings('.active').siblings('.active').addClass('active');
      $(this).siblings(


In [18]:
def gene(text):
  # text = "draw = function()"
  input_ids = tokenizer(text, return_tensors="pt").input_ids.cuda()

  # an.to_device("cpu")

  generated_ids = me.generate(input_ids, max_length=128)
  print(tokenizer.decode(generated_ids[0], skip_special_tokens=True))

In [17]:
me.train()

In [ ]:
torch.save(me, "/content/drive/MyDrive/bmo s/model.pt")

In [20]:
gene("move_player = function()")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


move_player = function() {
    $(".move-player").removeClass("move-player");
    $(".move-player").addClass("move-player-" + $("#" + $.cookie("move_player")).val());
    $(".move-player").val($("#" + $.cookie("move_player")).val());
    $(".move-player").change(function() {
      $(".move-player").removeClass("move-player");
      $(".move-player").addClass("move-


In [ ]:
# from transformers import AutoTokenizer, AutoModelForCausalLM
# tokenizer = AutoTokenizer.from_pretrained("Salesforce/codegen-350M-multi")
# model = AutoModelForCausalLM.from_pretrained("Salesforce/codegen-350M-multi")

# text = "def hello_world():"
# input_ids = tokenizer(text, return_tensors="pt").input_ids

# generated_ids = model.generate(input_ids, max_length=128)
# print(tokenizer.decode(generated_ids[0], skip_special_tokens=True))


# trying somethign with the comments

In [6]:
import pandas as pd

In [7]:
df = pd.read_csv("data.csv", index_col=False)
df['codes']

0       if start then\n  credits = {}\n  \n  \n  credi...
1       if start then\n  map = {}\n  \n  \n  map.name ...
2       if start then\n  player = {}\n  \n  \n  player...
3       function init()\n  screen:setPixelated("pixela...
4       function string:split(sep)\n  local sep, field...
                              ...                        
1232    /*\n  All of the code here is the level design...
1233    mower=object\n  \n  init=function()\n    ACC=0...
1234    sfx=object\n  \n  init=function()\n    sfxvol=...
1235    /*\n  Level section - Draws the 4x4 preview le...
1236    game=object\n  \n  init=function()\n    MAPXOF...
Name: codes, Length: 1237, dtype: object

In [130]:
def add_comment_code(string, spans):
  coms = [string[a:b] for (a,b) in spans]
  codes = []

  for i in range(len(spans)):
    start = spans[i][1]
    if(i==len(spans)-1):
      end = -1
    else:
      end = spans[i+1][0]
    codes.append(string[start:end])

  coms = [com.replace("//", " ").replace("\n"," ") for com in coms]
  
  return coms, codes

In [131]:
import re
l_comments = []
l_codes = []

for string in df['codes']:
  if(type(string) != str):
    continue
  
  l_spans = []
  prev = -1

  for i in re.finditer(r'\s*//.*\n',string):
    if(prev == i.span()[0]):
      temp = l_spans[-1][0]
      del l_spans[-1]
      l_spans.append((temp, i.span()[1]))
    else:
      l_spans.append((i.span()[0],i.span()[1]))
    prev = i.span()[1]

  com, cod = add_comment_code(string, l_spans)
  l_comments += com
  l_codes += cod

In [141]:
df = pd.DataFrame(columns= ["comments", "codes"], data = [[l_comments[i], l_codes[i]] for i in range(len(l_comments))])

In [150]:
df[df["comments"].str.contains('door')]

,comments,codes
4477,outdoor overlay (changes darkness accordi...,screen.setAlpha(0.0)\n if gameclock.h<9 the...
4479,indoor overlay,"col=rgbToString(fireplace.r,fireplace.g,fire..."
4490,outdoor position,"if gamemode==""indoor"" then farmer.x=31 far..."
4584,house front door target location,end\n collections.push(c)\n end
4612,At front doorway entry (from outdoor),"if active==""enter house from outdoor"" then..."
4613,At front doorway exit (from indoor),"if active==""exit to outdoor"" then\n g..."
4614,At rear doorway entry (from store room),"if active==""enter house from storeroom"" th..."
4615,At rear doorway exit (from indoor),"if active==""exit to storeroom"" then\n ..."
4764,"---- audio.playSound(""open_door"")",end\n \n \n if closed then\n ...
4775,check inside door,for d in doors\n local inside...
